<a href="https://colab.research.google.com/github/ESTEFANIALS/Guarderias/blob/main/Encuentra_tu_guarder%C3%ADa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Encuentra tu guardería

## Guarderías de la República Méxicana

### Equipo: 

*   Ana Lilia López Claro
*   Estefanía López Sernas

In [51]:
# Se importan las bibliotecas
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import geopy
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
import time
import math
import folium
from folium.plugins import HeatMap

In [56]:
df = pd.read_csv('https://raw.githubusercontent.com/ESTEFANIALS/Guarderias/main/base_guarderias')

In [60]:
df['Código Postal'] = df['Código Postal'].apply(lambda _: str(_))
informacion = []
for n in range(len(df)):
  informacion.append(df['Nombre de Unidad'][n] + ': ' + df['Calle'][n] + ' ' + str(df['Número Ext.'][n]) + ', Col. ' + df['Colonia'][n]  + ', Del. '+ df['Municipio'][n] + ', CP ' + str(df['Código Postal'][n]) + ' Horario: ' + df['Horario de Atención'][n] + ' Teléfono: ' + str(df['Teléfonos'][n]))
df["Información guardería"]= informacion

df_CDMX = df[df['Ciudad']== "Ciudad de México"]
df_CDMX =df_CDMX.dropna()

geo = Nominatim(user_agent='Myapp')

In [62]:
calle = input("Calle:")
num = input("Numero Exterior:")
dele = input("Delegación:")
num_opciones = input("Cuántas recomendaciones quieres:")

lugar = str(calle)+ ' ' +str(num)+str(', ')+str(dele)
posicion = 'El punto de referencia es: ' + str(lugar)

loc = geo.geocode(lugar)
print(loc)

coor= (loc.latitude,loc.longitude)
print(coor)


def f(x):
  return geopy.distance.distance(coor,(x["Latitud"],x["Longitud"]))

df_CDMX['Distancia']=df_CDMX.apply(f,axis=1)
#32df_CDMX['Distancia']=str(round(float(df_CDMX['Distancia'].split()[0]),2))
por_km1 = df_CDMX.sort_values('Distancia')

opciones1=por_km1.head(int(num_opciones))
dist1=opciones1['Distancia'].sum()
gasto1=dist1*20
#print('El costo de transporte de este recorrido es: $'+ str(round(float((str(gasto1)).split()[0]),2)))

seleccion1=opciones1[['Nombre de Unidad','Latitud','Longitud']]

nueva_fila = {'Nombre de Unidad':lugar, 'Latitud':loc.latitude, 'Longitud':loc.longitude}
mapa1=df_marks = seleccion1.append(nueva_fila, ignore_index=True)
mapa1= mapa1.set_index([np.arange(1,len(mapa1)+1)])

recomendadas = df_CDMX[df_CDMX['Nombre de Unidad'].isin(opciones1['Nombre de Unidad'])].sort_values('Distancia').set_index([np.arange(1,int(num_opciones)+1)])
print(recomendadas[['Distancia', 'Nombre de Unidad', 'Calle', 'Número Ext.', 'Código Postal', 'Municipio', 'Ciudad', 'Horario de Atención', 'Teléfonos']])

nueva_fila_r = {'Nombre de Unidad':lugar, 'Información guardería':posicion}
recomendadas = recomendadas.append(nueva_fila_r, ignore_index=True)

mapit1 = folium.Map(location = [19.4, -99.15], zoom_start=13)
for i in range (0,len(mapa1)):
  if mapa1.index[i] <= int(num_opciones):
    folium.Marker(
        location = [mapa1.Latitud.values[i], mapa1.Longitud.values[i]],
        popup= recomendadas['Información guardería'][i],
        icon=folium.Icon(color="green", icon="info-sign"),
    ).add_to(mapit1)
  else:
    folium.Marker(
        location = [mapa1.Latitud.values[i], mapa1.Longitud.values[i]],
        popup= recomendadas['Información guardería'][i],
    ).add_to(mapit1)

mapit1



Calle:Torre BBVA
Numero Exterior:510
Delegación:Cuauhtemoc
Cuántas recomendaciones quieres:10
Torre BBVA, Avenida Reforma, Colonia Nueva Anzures, Cuauhtémoc, Ciudad de México, México
(19.42265935, -99.17471273963955)
                 Distancia                                   Nombre de Unidad  \
1   0.26656115880172687 km     M-0008 Guardería Madres IMSS G-VIII Cuauhtémoc   
2   0.45175973043331147 km     G-0035 Guardería Ordinaria G - 0035 Cuauhtémoc   
3    0.5491395814472948 km  G-0041 Guardería Ordinaria G - 0041 Miguel Hid...   
4    0.5917917976337419 km                               U-0622 CENDI Hamelin   
5    0.5957243015855473 km                             U-0624 Colegio Cibeles   
6    1.1426568285418004 km     G-0003 Guardería Ordinaria G - 0003 Cuauhtémoc   
7    1.2307500146785335 km                        U-0702 Colegio del Angel II   
8    1.3312222741337787 km                           U-0704 Colegio Cibeles I   
9    1.4820467198697111 km                            